In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from Baudry_functions_v2 import *

# Assuming you have these functions already converted from R:
# simulate_central_scenario, join_policy_claim, time_slice_polclaim, RBNS_Train, RBNS_Test
# Also assuming lst_Date_slice is a list/array of datetime objects

# Simulate central scenario data
dt_PhoneData = simulate_central_scenario(1234)

# Join policy and claim data
dt_polclaim = join_policy_claim(dt_PhoneData,
                               date_pol_start="date_UW",
                               date_pol_end="date_lapse",
                               date_occur="date_occur")

# Create list of date slices (every ~30 days from 2016-01-01 to 2019-06-30)
lst_Date_slice = pd.date_range(start="2016-01-01", end="2019-06-30", freq='30D').to_pydatetime().tolist()

# Apply time slicing to policy-claim data
dt_polclaim = time_slice_polclaim(dt_polclaim, lst_Date_slice)

# Set valuation index and date
i = valuation = 10
t_i = lst_Date_slice[i-1]  # Python indexing starts at 0
delta = min(i, len(lst_Date_slice) - i + 1)

# Define model variables
RBNS_model_vars = [
    "clm_number",
    "pol_number",
    "j",
    "k",
    "date_pol_start",
    "date_occur",
    "date_report",
    "date_pay",
    "Cover",
    "claim_type",
    "Brand",
    "Model",
    "Price",
    "target"
]

# Create combined TRAIN dataset for k=1 and all valid j delay values
dt_RBNS_train = RBNS_Train(dt_polclaim, t_i, i, k=1, reserving_dates=lst_Date_slice, model_vars=RBNS_model_vars)

# Create combined TEST dataset for k=1 and all valid j delay values
dt_RBNS_test = RBNS_Test(dt_polclaim, t_i, delta, k=1, reserving_dates=lst_Date_slice, model_vars=RBNS_model_vars)

# Add flag columns to indicate train or test set
dt_RBNS_train['flgTrain'] = 1
dt_RBNS_test['flgTrain'] = 0

# Combine train and test dataframes
dt_All_RBNS = pd.concat([dt_RBNS_train, dt_RBNS_test], ignore_index=True)

# Optionally save to CSV
# dt_All_RBNS.to_csv("dt_All_RBNS.csv", index=False)


ValueError: All arrays must be of the same length